In [1]:
from pathlib import Path
from models.transfersdataset import TransfersDataset
from models.catboost import CatBoost

KeyboardInterrupt: 

In [ ]:
def pipeline(pos):
    td = TransfersDataset(Path.cwd()/'../prepped/final.csv')
    td.filter_postion(pos)
    td.encode()
    td.drop()
    display(td.data)
    m = CatBoost(
        target = 'marketval_0',
        features = ['age', 'season', 'window', 'loan', 'club_from_elo', 'club_to_elo', 'league_from_elo', 'league_to_elo'] + \
                td.data.columns[td.data.columns.str.contains('_-')].tolist(),
        data = td.data
    )
    m.train_test_split()
    m.scale()
    # m.tune_hp()
    m.train()
    m.inverse_scale()
    m.mae()
    m.mape()
    display(m.feature_importance())
    m.plot_predictions()
    display(m.top_n_predictions(20))
    

# Attackers

In [ ]:
pipeline('ATT')

MAE = 6951603.564249691
MAPE = 1.028672242964254


,name,age,season,country_from,league_from,club_from,country_to,league_to,club_to,window,fee,loan,marketval_0,marketval_0_pred,error
601,Suso,26,1.0,Italy,Serie A,Milan,Spain,LaLiga,Sevilla,1,21700000.0,0,20000000.0,1.995837e+07,4.163153e+04
48,I. Sarr,21,0.0,France,Ligue 1,Rennes,England,Premier League,Watford,1,30000000.0,0,24500000.0,2.455298e+07,5.298262e+04
1124,J. King,29,2.0,England,Premier League,Everton,England,Premier League,Watford,1,0.0,0,8000000.0,7.922098e+06,7.790166e+04
944,A. Modeste,32,1.0,Germany,Bundesliga,Köln,France,Ligue 1,Saint-Étienne,0,0.0,1,1500000.0,1.618385e+06,1.183849e+05
2329,R. Saponara,31,4.0,Italy,Serie A,Fiorentina,Italy,Serie A,Hellas Verona,1,0.0,0,1500000.0,1.286654e+06,2.133455e+05
162,N. Kalinić,31,0.0,Spain,LaLiga,Atlético Madrid,Italy,Serie A,Roma,1,2000000.0,1,6800000.0,7.250946e+06,4.509456e+05
426,K. Toko Ekambi,27,0.0,Spain,LaLiga,Villarreal,France,Ligue 1,Olympique Lyonnais,0,4000000.0,1,14000000.0,1.468286e+07,6.828617e+05
1298,C. Wood,30,2.0,England,Premier League,Burnley,England,Premier League,Newcastle United,0,30000000.0,0,10000000.0,9.204727e+06,7.952734e+05
485,A. Cerri,23,0.0,Italy,Serie A,Cagliari,Italy,Serie A,SPAL,0,0.0,1,3500000.0,4.325805e+06,8.258051e+05
952,S. Di Carmine,32,1.0,Italy,Serie A,Hellas Verona,Italy,Serie A,Crotone,0,0.0,1,1200000.0,2.047019e+06,8.470194e+05


# Midfielders

In [ ]:
pipeline('MID')

MAE = 4492317.126491903
MAPE = 0.6845550037763714


,name,age,season,country_from,league_from,club_from,country_to,league_to,club_to,window,fee,loan,marketval_0,marketval_0_pred,error
1926,G. Haraguchi,31,3.0,Germany,Bundesliga,Union Berlin,Germany,Bundesliga,Stuttgart,0,1000000.0,0,1000000.0,9.557922e+05,44207.792178
1701,J. Boëtius,28,3.0,Germany,Bundesliga,Mainz 05,Germany,Bundesliga,Hertha BSC,1,0.0,0,1500000.0,1.423044e+06,76956.401357
2277,L. Waldschmidt,27,4.0,Germany,Bundesliga,Wolfsburg,Germany,Bundesliga,Köln,1,0.0,1,3000000.0,2.906968e+06,93031.747822
308,K. Sema,25,0.0,England,Premier League,Watford,Italy,Serie A,Udinese,1,0.0,1,4000000.0,3.838762e+06,161237.626113
933,R. Snodgrass,33,1.0,England,Premier League,West Ham United,England,Premier League,West Bromwich Albion,0,110000.0,0,1500000.0,1.336260e+06,163739.980237
2209,Samú Costa,22,4.0,Spain,LaLiga,Almería,Spain,LaLiga,Mallorca,1,3000000.0,0,8000000.0,7.759959e+06,240040.733094
738,A. Aouchiche,18,1.0,France,Ligue 1,Paris Saint-Germain,France,Ligue 1,Saint-Étienne,1,0.0,0,10000000.0,9.696281e+06,303718.643246
450,A. Barák,25,0.0,Italy,Serie A,Udinese,Italy,Serie A,Lecce,0,0.0,1,8000000.0,8.321212e+06,321212.490647
1349,M. Cuisance,22,2.0,Germany,Bundesliga,Bayern München,Italy,Serie A,Venezia,0,2300000.0,0,4000000.0,3.618621e+06,381378.973952
1269,S. Khaoui,26,2.0,France,Ligue 1,Olympique Marseille,France,Ligue 1,Clermont,1,0.0,0,1500000.0,1.927138e+06,427137.659142


# Defenders

In [ ]:
pipeline('DEF')

MAE = 4075418.6463094805
MAPE = 0.5746280627085181


,name,age,season,country_from,league_from,club_from,country_to,league_to,club_to,window,fee,loan,marketval_0,marketval_0_pred,error
2399,D. Spence,23,4.0,England,Premier League,Tottenham Hotspur,Italy,Serie A,Genoa,0,0.0,1,8000000.0,8.144106e+06,1.441061e+05
805,A. Tripaldelli,21,1.0,Italy,Serie A,Sassuolo,Italy,Serie A,Cagliari,1,3000000.0,0,1000000.0,1.178231e+06,1.782309e+05
321,D. Lazović,28,0.0,Italy,Serie A,Genoa,Italy,Serie A,Hellas Verona,1,0.0,0,4000000.0,4.276243e+06,2.762425e+05
92,C. Smalling,29,0.0,England,Premier League,Manchester United,Italy,Serie A,Roma,1,3000000.0,1,16000000.0,1.631434e+07,3.143389e+05
1244,Sergio Escudero,31,2.0,Spain,LaLiga,Sevilla,Spain,LaLiga,Granada,1,0.0,0,1800000.0,1.462777e+06,3.372228e+05
253,J. Willems,25,0.0,Germany,Bundesliga,Eintracht Frankfurt,England,Premier League,Newcastle United,1,1000000.0,1,5500000.0,5.873346e+06,3.733460e+05
938,D. Vavro,24,1.0,Italy,Serie A,Lazio,Spain,LaLiga,Huesca,0,0.0,1,4000000.0,3.522546e+06,4.774541e+05
134,Denis Suárez,25,0.0,Spain,LaLiga,Barcelona,Spain,LaLiga,Celta de Vigo,1,12900000.0,0,9500000.0,9.011501e+06,4.884987e+05
1484,A. Hickey,20,3.0,Italy,Serie A,Bologna,England,Premier League,Brentford,1,14250000.0,0,25000000.0,2.549403e+07,4.940320e+05
222,I. Amadou,26,0.0,Spain,LaLiga,Sevilla,England,Premier League,Norwich City,1,0.0,1,5500000.0,4.983380e+06,5.166198e+05


# Goalies

In [ ]:
pipeline('GK')

MAE = 4480687.092467106
MAPE = 1.1385699854158484


,name,age,season,country_from,league_from,club_from,country_to,league_to,club_to,window,fee,loan,marketval_0,marketval_0_pred,error
165,G. Rulli,27,0.0,Spain,LaLiga,Real Sociedad,France,Ligue 1,Montpellier,1,1500000.0,1,8000000.0,7.873316e+06,1.266835e+05
216,R. Olsen,29,0.0,Italy,Serie A,Roma,Italy,Serie A,Cagliari,1,0.0,1,6500000.0,5.764805e+06,7.351954e+05
34,J. Cillessen,30,0.0,Spain,LaLiga,Barcelona,Spain,LaLiga,Valencia,1,35000000.0,0,12000000.0,1.053452e+07,1.465481e+06
527,Pepe Reina,37,0.0,Italy,Serie A,Milan,England,Premier League,Aston Villa,0,400000.0,1,650000.0,3.073151e+06,2.423151e+06
2187,Luís Maximiano,24,4.0,Italy,Serie A,Lazio,Spain,LaLiga,Almería,1,500000.0,1,5000000.0,7.687159e+06,2.687159e+06
908,M. Ryan,28,1.0,England,Premier League,Brighton,England,Premier League,Arsenal,0,0.0,1,6000000.0,8.778191e+06,2.778191e+06
2178,Y. Sommer,34,4.0,Germany,Bundesliga,Bayern München,Italy,Serie A,Inter Milan,1,6750000.0,0,5000000.0,8.046914e+06,3.046914e+06
2349,M. Sportiello,31,4.0,Italy,Serie A,Atalanta,Italy,Serie A,Milan,1,0.0,0,1500000.0,5.250530e+06,3.750530e+06
1718,I. Provedel,28,3.0,Italy,Serie A,Spezia,Italy,Serie A,Lazio,1,2550000.0,0,12000000.0,7.092521e+06,4.907479e+06
345,Juan Soriano,21,0.0,Spain,LaLiga,Sevilla,Spain,LaLiga,Leganés,1,0.0,1,1600000.0,6.624552e+06,5.024552e+06
